In [ ]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#display all columns in df
pd.set_option('display.max_columns', None)

In [ ]:
# Filter LCT by chamber
def select_chamber(LCT_df, endcap='all', station='all', ring='all', chamber='all'):
    if (endcap != 'all'):
        LCT_df = LCT_df.query('endcap==@endcap')
    if (station != 'all'):
        LCT_df = LCT_df.query('station==@station')
    if (ring != 'all'):
        LCT_df = LCT_df.query('ring==@ring')
    if (chamber != 'all'):
        LCT_df = LCT_df.query('chamber==@chamber')
    return LCT_df
import pandas as pd

# Filter dataframe and saves only events with exactly one data and one emul LCT in each chamber
def filterSingleLCTs(LCT_df): 
    # Filter data and emulator rows separately
    LCT_df_data = LCT_df[LCT_df['is_data'] == True]
    LCT_df_emul = LCT_df[LCT_df['is_emul'] == True]

    # Group data and emul by Event, Endcap, Station, Ring, and Chamber
    grouped_data = LCT_df_data.groupby(['Event', 'endcap', 'station', 'ring', 'chamber'])
    grouped_emul = LCT_df_emul.groupby(['Event', 'endcap', 'station', 'ring', 'chamber'])

    # Lists to store filtered data
    filtered_list = []

    # Iterate through each group
    for group_name, group_data in grouped_data:
        try:
            group_emul = grouped_emul.get_group(group_name)
        except KeyError:
            # Skip groups where no emulator LCT exists
            continue

        # Check if there is exactly one LCT for both data and emul in the group
        if len(group_data) == 1 and len(group_emul) == 1:
            filtered_list.append(group_data)
            filtered_list.append(group_emul)

    # Concatenate the filtered dataframes
    filtered_df = pd.concat(filtered_list)

    return filtered_df

# Read DF from root file
def read_root_file(filename):
    file = uproot.open(filename)
    dataframes = {}
    for key in file.keys():
        if isinstance(file[key], uproot.behaviors.TTree.TTree):
            tree = file[key]
            df = tree.arrays(library="pd")
            dataframes[key] = df.reset_index()
    return dataframes

# Function for matching Emulator and Data LCTs
# This function needs an optimization. On my M2 laptop 10000 events take 1.5 minutes to process. My guess if use numpy and vectorize the function it will be >10 times faster.
# This function creates a new column 'match' in LCT_df. Each matchet pair has it's unique number. If LCT has no match it has 0 in 'match' column.
def matchLCTs(LCT_df):
    LCT_df_data = LCT_df[LCT_df['is_data'] == True]
    LCT_df_emul = LCT_df[LCT_df['is_emul'] == True]
    LCT_matched = LCT_df.copy()
    LCT_matched['match'] = 0
    match_count = 1

    # Group data and emul by event, endcap, station, ring and chamber
    grouped_data = LCT_df_data.groupby(['Event', 'endcap', 'station', 'ring', 'chamber'])
    grouped_emul = LCT_df_emul.groupby(['Event', 'endcap', 'station', 'ring', 'chamber'])
    # Iterate through each group
    for group_name, group_data in grouped_data:
        try:
            group_emul = grouped_emul.get_group(group_name)
        except KeyError:
            # Case if emul has no LCT in same Event/endcap/station/ring/chamber and data has
            continue
        # Match data and emul within the group
        for i in group_data.index:
            for j in group_emul.index:
                conditions = [
                    group_data.loc[i, 'quality'] == group_emul.loc[j, 'quality'],
                    group_data.loc[i, 'keyWG'] == group_emul.loc[j, 'keyWG'],
                    group_data.loc[i, 'strip'] == group_emul.loc[j, 'strip'],
                    group_data.loc[i, 'quadStrip'] == group_emul.loc[j, 'quadStrip'],
                    group_data.loc[i, 'eightStrip'] == group_emul.loc[j, 'eightStrip'],
                    group_data.loc[i, 'stripType'] == group_emul.loc[j, 'stripType'],
                    group_data.loc[i, 'bend'] == group_emul.loc[j, 'bend'],
                    group_data.loc[i, 'slope'] == group_emul.loc[j, 'slope'],
                    group_data.loc[i, 'pattern'] == group_emul.loc[j, 'pattern'],
                    group_data.loc[i, 'run3pattern'] == group_emul.loc[j, 'run3pattern']
                ]
                if all(conditions):
                    LCT_matched.loc[i, 'match'] = match_count
                    LCT_matched.loc[j, 'match'] = match_count
                    match_count += 1

    return LCT_matched


In [ ]:
df = read_root_file("data/out_GEMCSCTriggerPrimitivesReader.root")
ALCT_df = df['GEMCSCTriggerPrimitivesReader/ALCT_tree;1']
CLCT_df = df['GEMCSCTriggerPrimitivesReader/CLCT_tree;1']
LCT_df = df['GEMCSCTriggerPrimitivesReader/LCT_tree;1']

In [ ]:
def plot_LCT_par_hist(LCT_df, par, endcap='all', station='all', ring='all', chamber='all'):
    LCT_df = select_chamber(LCT_df, endcap, station, ring, chamber)
    LCT_df_data = LCT_df.query('is_data==1')
    LCT_df_emul = LCT_df.query('is_emul==1')
    plt.figure(figsize=(14, 3))
    bins = max(len(LCT_df_data[par].unique()), len(LCT_df_emul[par].unique()))
    range = (min(LCT_df_data[par].min(), LCT_df_emul[par].min()), max(LCT_df_data[par].max(), LCT_df_emul[par].max()))
    plt.hist(LCT_df_data[par], bins=bins, range=range, histtype='bar', alpha = 0.7, label='data', align='mid', rwidth=0.8, color='red')
    plt.hist(LCT_df_emul[par], bins=bins, range=range, histtype='bar', alpha = 0.7, label='emul', align='mid', rwidth=0.6, color='blue')
    # plt.xticks(np.arange(0.5, 9.5, 1), ('0', '1', '2', '3', '4', '5', '6', '7', '8'))
    plt.xlabel('Patameter: ' + par + ', Endcap: ' + str(endcap) + ', Station: ' + str(station) + ', Ring: ' + str(ring) + ', Chamber: ' + str(chamber))
    plt.ylabel('Entries')
    plt.title('LCT Parameter Histogram')
    plt.legend()
    plt.show()


In [ ]:
def plot_matched_LCTs_count(LCT_df, endcap='all', station='all', ring='all', chamber='all'):
    LCT_matched = LCT_df.copy()
    LCT_matched = select_chamber(LCT_matched, endcap, station, ring, chamber)
    LCT_matched['match_count'] = LCT_matched['match'].apply(lambda x: 1 if x > 0 else 0)
    LCT_matched_data = LCT_matched[LCT_matched['is_data'] == True]
    LCT_matched_emul = LCT_matched[LCT_matched['is_emul'] == True]
    plt.figure(figsize=(4, 3))
    plt.hist(LCT_matched_data['match_count'],bins = 2, range = (0, 1),  histtype='bar', alpha=0.7, label='data', align='mid', rwidth=0.8)
    plt.hist(LCT_matched_emul['match_count'],bins = 2, range = (0, 1), histtype='bar', alpha=0.7, label='emul', align='mid', rwidth=0.6)
    plt.xticks([0.25, 0.75], ['No match', 'Match'], )
    plt.xlabel('Matched LCTs count,\n Endcap: ' + str(endcap) + ', Station: ' + str(station) + ', Ring: ' + str(ring) + ', Chamber: ' + str(chamber))
    plt.ylabel('Entries')
    plt.legend()
    plt.show()

In [ ]:
def plot_heatmap(LCT_matched):
    # Calculate the ratio of 'data with match' divided by 'data no match' for each chamber
    ratio_data = LCT_matched.groupby(['endcap', 'station', 'ring', 'chamber']).apply(
        lambda group: group[group['is_data']]['match'].apply(lambda x: 1 if x > 0 else 0).sum() / (group[group['is_data']]['match'].apply(lambda x: 1 if x == 0 else 0).sum() + group[group['is_data']]['match'].apply(lambda x: 1 if x > 0 else 0).sum())
    )

    # Create a DataFrame with the calculated ratios
    ratio_df = pd.DataFrame({
        'Chamber': ratio_data.index.get_level_values('chamber'),
        'Endcap': ratio_data.index.get_level_values('endcap'),
        'Station': ratio_data.index.get_level_values('station'),
        'Ring': ratio_data.index.get_level_values('ring'),
        'Ratio': ratio_data.values
    })

    # Pivot the DataFrame for easy plotting
    ratio_pivot = ratio_df.pivot_table(index=['Chamber'], columns=['Endcap', 'Station', 'Ring'], values='Ratio')

    # Convert the ratio_pivot DataFrame to a 2D NumPy array
    data_array = ratio_pivot.to_numpy()

    # Set up the Matplotlib figure and axis
    fig, ax = plt.subplots(figsize=(14, 8))

    # Create a heatmap using Matplotlib's pcolor
    cax = ax.pcolor(data_array, cmap='viridis', edgecolors='w', linewidths=0.5)

    # Add colorbar
    cbar = plt.colorbar(cax, label='Data with Match / All Data')

    # Set axis labels and title
    ax.set_title('2D Heatmap: Data with Match / All Data')
    ax.set_xlabel('Endcap - Station - Ring')
    ax.set_ylabel('Chamber')

    # Add chamber labels on the y-axis
    ax.set_yticks(range(len(ratio_pivot.index)))
    ax.set_yticklabels(ratio_pivot.index)

    # Add endcap-station-ring labels on the x-axis
    ax.set_xticks(range(len(ratio_pivot.columns)))
    ax.set_xticklabels(ratio_pivot.columns, rotation=90, ha='right')

    plt.show()
